In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_name = "google/flan-t5-large"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving java_qa_dataset.json to java_qa_dataset.json


In [ ]:
def preprocess(batch):
    input_texts = ["Java Question: " + q for q in batch["instruction"]]
    target_texts = batch["output"]
    return tokenizer(
        input_texts,
        text_target=target_texts,
        truncation=True,
        padding="max_length",
        max_length=256,
    )


In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("json", data_files="java_qa_dataset.json")["train"]


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Split dataset: 80% train, 10% eval, 10% test
train_test = dataset.train_test_split(test_size=0.2, seed=42)
eval_test = train_test["test"].train_test_split(test_size=0.5, seed=42)

train_data = train_test["train"]
eval_data = eval_test["train"]
test_data = eval_test["test"]


In [ ]:
tokenized_train = train_data.map(preprocess, batched=True)
tokenized_eval = eval_data.map(preprocess, batched=True)
tokenized_test = test_data.map(preprocess, batched=True)


Map:   0%|          | 0/1588 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./flan-t5-lora-java-results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-4,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer
)


<ipython-input-8-dc114e07342d>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()
model.save_pretrained("./flan-t5-lora-java-results")


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.062700
20,0.109000
30,0.103400
40,0.078000
50,0.107000
60,0.063800
70,0.086600
80,0.093400
90,0.143900
100,0.126400


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentran

Step,Training Loss
10,0.062700
20,0.109000
30,0.103400
40,0.078000
50,0.107000
60,0.063800
70,0.086600
80,0.093400
90,0.143900
100,0.126400


In [ ]:
from transformers import AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

java_questions = [
    "What is the difference between final, finally and finalize in Java?",
    "What is method overloading in Java?",
    "What is garbage collection?",
    "Explain static vs instance variables.",
    "What is the use of final keyword?"
]

for question in java_questions:
    input_text = "Java Question: " + question
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            temperature=0.7,
            repetition_penalty=1.2
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n🧠 Q: {question}\n✅ A: {answer}")


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



🧠 Q: What is the difference between final, finally and finalize in Java?
✅ A: final is a keyword to define constants or prevent overriding, finally is used in exception handling to execute code after try-catch, and finalize is a method called by the garbage collector before destroying the object.

🧠 Q: What is method overloading in Java?
✅ A: Method overloading is when multiple methods in a class have the same name but different parameters.

🧠 Q: What is garbage collection?
✅ A: Garbage collection is the process of collecting and disposing of garbage in a garbage collector.

🧠 Q: Explain static vs instance variables.
✅ A: Static variables belong to the class and are shared among all objects, while instance variables are unique to each object.

🧠 Q: What is the use of final keyword?
✅ A: The final keyword is used to refer to the current instance of the class.


In [ ]:
!pip install evaluate rouge_score



  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=286abef6dd40abf5614f6b241eab0d3db6e42e1518ba66c2262bc778c5aa0fba
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [ ]:
# Sample questions (can use test_data here if needed)
sample_questions = [
    "What is a constructor in Java?",
    "What is method overloading in Java?",
    "What is garbage collection?",
]

reference_answers = [
    "A constructor is a special method used to initialize objects in Java.",
    "Method overloading is when multiple methods have the same name but different parameters.",
    "Garbage collection is the process of automatically reclaiming memory used by unreferenced objects."
]

predicted_answers = []
for q in sample_questions:
    inputs = tokenizer("Java Question: " + q, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predicted_answers.append(answer)

# Evaluate
bleu_score = bleu.compute(predictions=predicted_answers, references=[[r] for r in reference_answers])
rouge_score = rouge.compute(predictions=predicted_answers, references=reference_answers)

print("\n📊 Evaluation Metrics:")
print(f"BLEU Score     : {bleu_score['bleu']:.4f}")
print(f"ROUGE-1        : {rouge_score['rouge1']:.4f}")
print(f"ROUGE-2        : {rouge_score['rouge2']:.4f}")
print(f"ROUGE-L        : {rouge_score['rougeL']:.4f}")


📊 Evaluation Metrics:
BLEU Score     : 0.4548
ROUGE-1        : 0.6545
ROUGE-2        : 0.5847
ROUGE-L        : 0.6545


In [ ]:
!pip install gradio --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch


def answer_java_question(question):
    prompt = "Answer this Java programming question clearly:\nJava Question: " + question
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.8,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.2
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Theme Customization ---
theme = gr.themes.Default(
    primary_hue="indigo",
    secondary_hue="blue"
).set(
    body_background_fill="#f8fafc",
    container_radius="xl",
    input_background_fill="white",
    input_border_color="#cbd5e1",
    input_border_width="1px",
    button_primary_background_fill="indigo",
    block_title_text_weight="bold",
    block_title_text_size="xl"
)

# --- Interface Layout ---
with gr.Blocks(theme=theme, title="Java Programming Q&A Bot", css="""
    #main-container { max-width: 800px; margin: auto; }
    #footer {
        margin-top: 2em;
        color: #94a3b8;
        font-style: italic;
        text-align: center;
        font-size: 0.9em;
    }
""") as demo:
    gr.Image("https://upload.wikimedia.org/wikipedia/en/3/30/Java_programming_language_logo.svg", width=80)
    gr.Markdown("## 💡 Java Programming Q&A Bot")
    gr.Markdown("🔍 Ask any **Java-related** programming question and get accurate, AI-powered answers instantly!")

    with gr.Row():
        with gr.Column(scale=3):
            input_box = gr.Textbox(
                lines=3,
                placeholder="Enter your Java programming question here...",
                label="❓ Your Java Question"
            )
            submit_btn = gr.Button("Get Answer 💬")

        with gr.Column(scale=2):
            output_box = gr.Textbox(
                label="✅ AI Response",
                lines=6,
                interactive=False
            )

    # Example questions
    examples = gr.Examples(
        examples=[
            "What is a constructor in Java?",
            "What is the difference between abstract class and interface?",
            "How does garbage collection work in Java?",
            "What is the use of 'final' keyword in Java?",
            "Explain static vs instance variables in Java."
        ],
        inputs=[input_box]
    )

    # Footer
    gr.Markdown(
        "💬 Powered by `flan-t5-large` fine-tuned with LoRA · Built with ❤️ by Thivya Dhanasegaran",
        elem_id="footer"
    )

    # Connect input/output
    submit_btn.click(fn=answer_java_question, inputs=input_box, outputs=output_box, show_progress="full")

# --- Launch App ---
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3b463f47a55591ea64.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
